In [1]:
import re
import pandas as pd
import numpy as np

In [2]:
# Remove the special characters and images from a column. 
def removeSpecial(df_column):
    return df_column.astype(str).replace(r"[^\w\s,\.:!\/\&\-']+", "", regex=True).str.strip()

In [3]:
info_df = pd.read_csv('raw_info_df.csv').loc[:, ['jobTitle', 'description', 'employmentType', 'url', 'hiringOrganization.name', 'baseSalary.raw', 'baseSalary.valueMax', 'jobLocation.raw',]]

info_df.drop_duplicates(subset=['description'], inplace=True, ignore_index=True)

In [4]:
# These are the columns that can go through the removeSpecial function at the beginning.  
# The other columns are not yet ready - it's helpful to have the dollar symbol in front of a salary.
tidyColumnsBegin = ['description', 'employmentType', 'hiringOrganization.name', 'jobLocation.raw']

for col in tidyColumnsBegin:
    info_df[col] = removeSpecial(info_df[col])

In [5]:
# Updates for info_df['employmentType'] 
# Updated values to ensure value consistency. 
typeMask = info_df['employmentType'].str.contains("full", case=False)
info_df.loc[typeMask,['employmentType']] = "Full-Time"

# Removed values that do not align with the column title.
validEmploy = ['Remote', 'Full-Time', 'Freelance/Independent Contract']
info_df['employmentType'] = info_df['employmentType'].where(info_df['employmentType'].isin(validEmploy), np.nan)

In [6]:
# Splits the title to remove the location information. 
# The location information was checked against the df['jobLocation.raw'] rows with missing information, but no additional info was found.
info_df['jobTitle'] = info_df['jobTitle'].str.split('(', n=1, expand=True)[0]
info_df['jobTitle'] = removeSpecial(info_df['jobTitle'])

# Updated values for info_df['jobLocation.raw']
locMask = info_df['jobLocation.raw'].str.contains("Remote", case=False, na=None, regex=True)
info_df.loc[locMask,['jobLocation.raw']] = "Remote"

info_df['hiringOrganization.name'] = info_df['hiringOrganization.name'].str.replace("About", "", case=False, regex=True).str.strip()

In [7]:
# Extracting the salaries in the baseSalary column.
# "\$?\d{2,3},?\d{3}" For standard salary figures, an optional dollar sign and an optional comma, being 5-6 digits total
# "\$?\d{2,3}[kK]" For salaries expressed with k or K, digits with an optional dollar sign. 

info_df['baseSalary.raw'] = info_df['baseSalary.raw'].astype(str)
patternSalary = r"(\$?\d{2,3},?\d{3}|\$?\d{2,3}[kK])"

# Keep index to create ['level_0'], it is the index foreign index to info_df.
salary_df = info_df['baseSalary.raw'].str.extractall(patternSalary).reset_index(drop=False)

# Standardized extracted values, located in salary_df[0] 
salary_df[0] = salary_df[0].str.lower().replace({'401k': np.nan, 'k': '000', ',': ''}, regex=True)
salary_df.dropna(axis=0, inplace=True)

salary_df[0] = removeSpecial(salary_df[0])
salary_df[0] = salary_df[0].astype('int64')

minSalary = salary_df.groupby('level_0')[0].min().reset_index(name='minSalary')
maxSalary = salary_df.groupby('level_0')[0].max().reset_index(name='maxSalary')

salary_df = pd.merge(minSalary, maxSalary, on='level_0', how='outer').set_index('level_0')

In [8]:
# Joining the two dfs adds additional columns on the indexes of info_df and salary_df
df = info_df.join(salary_df).copy()
df.drop(['baseSalary.raw', 'baseSalary.valueMax'], axis=1, inplace=True)
df.columns = ['jobTitle', 'description', 'employmentType', 'url', 'hiringOrganization', 'jobLocation', 'minSalary', 'maxSalary']

# Reordered columns
df = df[['jobTitle', 'description', 'jobLocation', 'minSalary', 'maxSalary', 'hiringOrganization', 'employmentType', 'url', ]]
df.replace('nan', np.nan, inplace=True)

In [9]:
df['jobTitle'] = df['jobTitle'].str.replace(r"Sr\W", "Senior ", case=False, regex=True)\
                                .str.replace(r" - | & |/", " ", regex=True)\
                                .str.replace(r"[,]", "", regex=True)\
                                .str.replace(r"\s{2,}", " ", regex=True).str.strip()\
                                .replace("", "Data Scientist")# Adding the most frequent jobTitle for any missing jobTitles.

In [10]:
df.to_csv('metadataCleaned.csv', index=False)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 507 entries, 0 to 506
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   jobTitle            507 non-null    object 
 1   description         507 non-null    object 
 2   jobLocation         497 non-null    object 
 3   minSalary           225 non-null    float64
 4   maxSalary           225 non-null    float64
 5   hiringOrganization  244 non-null    object 
 6   employmentType      54 non-null     object 
 7   url                 507 non-null    object 
dtypes: float64(2), object(6)
memory usage: 31.8+ KB


In [12]:
df.head(10)

,jobTitle,description,jobLocation,minSalary,maxSalary,hiringOrganization,employmentType,url
0,Senior Scientist,"At Element Biosciences, we are passionate abou...","San Diego, CA",126000.0,165000.0,Element Biosciences,Full-Time,https://builtin.com/job/sr-scientist/2179388
1,Data Scientist Finance,About Us:\n\nLive experiences help people cros...,Remote,173000.0,205000.0,Gametime United,NaN,https://builtin.com/job/data-scientist-finance...
2,Air Quality Modeling Scientist,PSE Healthy Energy is seeking a full-time air ...,Remote,75000.0,97500.0,PSE Healthy Energy,Remote,https://builtin.com/job/air-quality-modeling-s...
3,Data Scientist,We are interested in every qualified candidate...,"Chicago, IL",NaN,NaN,Enova,NaN,https://builtin.com/job/data-scientist-hybrid/...
4,Marketing Data Research Specialist,Company Overview\n\nAdCellerant is an award-wi...,Greater Denver Area,55000.0,60000.0,AdCellerant,NaN,https://builtin.com/job/data/marketing-data-re...
5,Manager Data Science,WeAreCrowdStrike and our mission is to stop br...,"Wilmington, NC",130000.0,205000.0,NaN,NaN,https://builtin.com/job/manager-data-science/2...
6,Revenue Operations Analyst,Upgrade is a fintech company that provides aff...,"Atlanta, GA",NaN,NaN,NaN,NaN,https://builtin.com/job/revenue-operations-ana...
7,Data Scientist,Shape the future with intelligence and creativ...,"San Francisco, CA",90000.0,105000.0,The Creative Data Company,NaN,https://builtin.com/job/data-scientist/2272211
8,Senior Lead Data Scientist,"Headquartered in Boston, Forward Financing has...",Remote,55000.0,55000.0,NaN,NaN,https://builtin.com/job/senior-data-scientist/...
9,Solaria Labs Data Science Co-op,Pay Philosophy\nThe typical starting salary ra...,"Solaria Labs, Boston, MA",33000.0,83000.0,Liberty Mutual Insurance,NaN,https://builtin.com/job/solaria-labs-data-scie...


In [13]:
df.nunique()

jobTitle              320
description           507
jobLocation           145
minSalary             134
maxSalary             127
hiringOrganization    234
employmentType          3
url                   507
dtype: int64